In [ ]:
import os
from importlib import reload
import spacy
from gensim import models
import tensorflow as tf

In [ ]:
BEHIND_THE_WORDS_DIR = "./"
DATA_DIR = os.path.join(BEHIND_THE_WORDS_DIR, "data")

In [ ]:
_nlp = spacy.load("en_core_web_sm")

In [ ]:
from utils.load_word2vec import load_word2vec

w2v_model_path = os.path.join(BEHIND_THE_WORDS_DIR, "metaphor/data/gensim/word2vec-google-news-300.gz")
word2vec = load_word2vec(w2v_model_path, "http://127.0.0.1:7070")

In [ ]:
from utils.dir import get_latest_file_in_dir, get_filename

LATEST_CNN_MODEL = get_latest_file_in_dir("./models/cnn/")
CNN_MODEL_NAME = get_filename(LATEST_CNN_MODEL)
model_cnn = tf.keras.saving.load_model(LATEST_CNN_MODEL, compile=True, safe_mode=True)
print(f"Using CNN Model {LATEST_CNN_MODEL}")

MAX_TOKEN_LENGTH = 384

def process_text(text, max_token_length=None):
  if max_token_length == None:
    max_token_length = MAX_TOKEN_LENGTH
    
  doc = _nlp(text)
  words = [token.lower_ for token in doc]
  embeddings = word2vec.get_vec(words).tolist()[:MAX_TOKEN_LENGTH]
  padding = [[0] * 300] * (MAX_TOKEN_LENGTH - len(embeddings))

  return embeddings + padding

def predict_cnn(model_cnn, text):
  data = np.array([process_text(text)])
  data = data.reshape(-1, *data.shape[1:], 1)

  return model_cnn.predict(data, verbose=None).tolist()[0][0]

In [ ]:
import loader.JSONLDataloader as JSONLDataloader
reload(JSONLDataloader)

dataset_jsonl_dataloader = JSONLDataloader.JSONLDataloader(DATA_DIR)

In [ ]:
import preprocessor.CNNPredictionPreprocessor as CNNPredictionPreprocessor

reload(CNNPredictionPreprocessor)

cnn_prediction_preprocessor = CNNPredictionPreprocessor.CNNPredictionPreprocessor(process_text, dataset_jsonl_dataloader, _dir = DATA_DIR, in_folders=["rf-cnn"], out_folders=["rf-cnn", "cnn-prediction", CNN_MODEL_NAME])

In [ ]:
cnn_prediction_preprocessor.preprocess("essayforum", model_cnn)
cnn_prediction_preprocessor.preprocess("own", model_cnn)